In [1]:
import datetime

import pymysql
from flask import Flask, request,make_response, jsonify,render_template
from flask_cors import CORS
from werkzeug.utils import secure_filename

import os
import random

In [2]:
WORK_DIR = os.getcwd()
SERVER_ADDR = 'http://localhost:7777/'

## SQL部分

In [3]:
def execute_sql(sql):
    db = pymysql.connect("localhost","root","cmq19950520","Chatroom" )
    cursor = db.cursor()
    cursor.execute(sql)
    result = cursor.fetchall()
    db.commit()
    db.close()
    return result


In [4]:
#用户表，用户名和密码
def init_user_table():
    sql = """CREATE TABLE USER (
             user_id INT UNSIGNED AUTO_INCREMENT,
             username VARCHAR(20) NOT NULL UNIQUE,
             nickname VARCHAR(20),
             password VARCHAR(20) NOT NULL,
             avatar VARCHAR(100) NOT NULL DEFAULT 'images/default.jpg',
             PRIMARY KEY (user_id)
             )ENGINE=InnoDB DEFAULT CHARSET=utf8;"""
    execute_sql(sql)
def show_user_table():
    sql = """SELECT * FROM USER"""
    result = execute_sql(sql)
    return result

def insert_user_item(username,password):
    sql = """INSERT INTO USER(username,password)
             VALUES('%s','%s');"""%(username,password)
    execute_sql(sql)
#除了主键，皆可更新
def update_password(username,password):
    sql = """UPDATE USER SET password='%s' 
             WHERE username='%s';"""%(password,username)
    execute_sql(sql)
def update_avatar(username,avatar):
    sql = """UPDATE USER SET avatar='%s' 
             WHERE username='%s';"""%(avatar,username)
    execute_sql(sql)
def update_nickname(username,nickname):
    sql = """UPDATE USER SET nickname='%s' 
             WHERE username='%s';"""%(nickname,username)
    execute_sql(sql)
def get_user_by_name(username):
    sql = """SELECT * FROM USER WHERE username='%s';"""%(username)
    result = execute_sql(sql)
    return result
def get_user_by_id(user_id):
    sql = """SELECT * FROM USER WHERE user_id='%s';"""%(user_id)
    result = execute_sql(sql)
    return result


In [5]:
#聊天记录表，谁向谁发送了什么信息
def init_chat_history_table():
    sql = """CREATE TABLE CHAT_HISTORY (
             id INT UNSIGNED AUTO_INCREMENT,
             user_id INT UNSIGNED NOT NULL,
             target_id INT UNSIGNED NOT NULL,
             target_type VARCHAR(20) NOT NULL DEFAULT 'chatroom' COMMENT 'user or chatroom',
             content TEXT NOT NULL,
             time DATETIME NOT NULL,
             PRIMARY KEY ( id )
             )ENGINE=InnoDB DEFAULT CHARSET=utf8;"""
    execute_sql(sql)
def show_chat_history_table():
    sql = """SELECT * FROM CHAT_HISTORY"""
    result = execute_sql(sql)
    return result
def insert_chat_history_item(user_id,target_id,target_type,content,time='now'):
    if time=='now':
        time = datetime.datetime.now().__format__("%Y-%m-%d %H:%M")
        
    sql = """INSERT INTO CHAT_HISTORY(user_id,target_id,content,time)
             VALUES('%s','%s','%s','%s');"""%(user_id,target_id,content,time)
    execute_sql(sql)
    
def check_chat_history(target_id,target_type,start_time,end_time):
    sql = """SELECT * FROM CHAT_HISTORY WHERE target_id='%s' AND target_type='%s' AND
             time > '%s' AND time < '%s' ; """%(target_id,target_type,start_time,end_time)
    result = execute_sql(sql)
    return result

In [6]:
#聊天室表，谁加入了哪个聊天室
def init_chatroom_table():
    sql = """CREATE TABLE CHATROOM_MEMBER(
             room_id INT UNSIGNED NOT NULL,
             user_id INT UNSIGNED NOT NULL,
             PRIMARY KEY ( room_id,user_id )
             )ENGINE=InnoDB DEFAULT CHARSET=utf8;"""
    execute_sql(sql)
def show_chatroom_table():
    sql = """SELECT * FROM CHATROOM_MEMBER"""
    result = execute_sql(sql)
    return result
def insert_chatroom_item(room_id,user_id):
    sql = """INSERT INTO CHATROOM_MEMBER(room_id,user_id)
             VALUES('%s','%s')"""%(room_id,user_id)
    execute_sql(sql)
def get_chatroom_item(room_id,user_id):
    sql = """SELECT * FROM CHATROOM_MEMBER WHERE room_id='%s' AND user_id='%s';"""%(room_id,user_id);
    result = execute_sql(sql);
    return result;
def delete_chatroom_item(room_id,user_id):
    sql = """DELETE FROM CHATROOM_MEMBER WHERE room_id='%s' AND user_id='%s';"""%(room_id,user_id)
    execute_sql(sql)


def get_chatroom_user(room_id):
    sql = """SELECT user_id FROM CHATROOM_MEMBER WHERE room_id = %s"""%(str(room_id))
    result = execute_sql(sql)
    return result

In [7]:
# #每一个chatroom的信息，包括id，名称，
# def init_chatroom_info_table():
#     sql = """CREATE TABLE CHATROOM_INFO (
#              room_id INT UNSIGNED NOT NULL,
#              name VARCHAR(20) NOT NULL,
#              avatar VARCHAR(100) NOT NULL DEFAULT 'images/default.jpg',
#              PRIMARY KEY (room_id)
#              )ENGINE=InnoDB DEFAULT CHARSET=utf8;"""
#     execute_sql(sql)
# def show_chatroom_info_table():
#     sql = """SELECT * FROM CHATROOM_INFO"""
#     result = execute_sql(sql)
#     return result

# def insert_user_item(username,password):
#     sql = """INSERT INTO CHATROOM_INFO(username,password)
#              VALUES('%s','%s');"""%(username,password)
#     execute_sql(sql)
# #除了主键，皆可更新
# def update_password(username,password):
#     sql = """UPDATE USER SET password='%s' 
#              WHERE username='%s';"""%(password,username)
#     execute_sql(sql)
# def update_avatar(username,avatar):
#     sql = """UPDATE USER SET avatar='%s' 
#              WHERE username='%s';"""%(avatar,username)
#     execute_sql(sql)
# def update_nickname(username,nickname):
#     sql = """UPDATE USER SET nickname='%s' 
#              WHERE username='%s';"""%(nickname,username)
#     execute_sql(sql)
# def get_user_by_name(username):
#     sql = """SELECT * FROM USER WHERE username='%s';"""%(username)
#     result = execute_sql(sql)
#     return result
# def get_user_by_id(user_id):
#     sql = """SELECT * FROM USER WHERE user_id='%s';"""%(user_id)
#     result = execute_sql(sql)
#     return result


In [10]:
# def init_all_tables():
#     init_chat_history_table()
#     init_chatroom_table()
#     init_user_table()
# init_all_tables()

/Users/cmq2525/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


## 后台逻辑

In [8]:
#登录
def register(username,password):
    result = get_user_by_name(username)
    if len(result) == 0:
        insert_user_item(username,password)
        return 'suc'
    else:
        return 'already registered'
#登录
def login(username,password):
    result = get_user_by_name(username)
    if result and result[0][3] == password:
        return result[0][0]
    else:
        return -1

In [9]:
#加入聊天室
def create_chatroom(user_id):
    #找到一个没用过的room_id
    room_id = random.randint(0,10000)
    while(len(get_chatroom_user(room_id))!=0):
        room_id = random.randint(0,10000)
    insert_chatroom_item(room_id,user_id)
    return room_id
        
def join_chatroom(room_id,user_id):
    if len(get_chatroom_item(room_id,user_id))==0:
        insert_chatroom_item(room_id,user_id)
    

In [10]:
#接收聊天信息
def chat_in_chatroom(room_id,user_id,content):
    insert_chat_history_item(room_id,user_id,'chatroom',content)
#刷新聊天信息
def refresh_chat(target_id,target_type,start_time):
    def process_chat_history(chat_history):
        conv_id,user_id,target_id,target_type,content,time = chat_history
        time = time.strftime("%Y-%m-%d %H:%M:%S")
        user_item = get_user_by_id(user_id)
        #如果有nickname就返回nickname，否则返回username
        username = user_item[0][1]
        #只需要知道作者，时间，内容
        return [username,content,time]
    end_time = datetime.datetime.now()
    chat_content = check_chat_history(target_id,target_type,start_time,end_time)
    if len(chat_content):
        chat_content = list(map(process_chat_history,chat_content))
    return chat_content,end_time
    

In [34]:
app = Flask(__name__,
            static_url_path='', 
            static_folder='dist',
            template_folder='dist')
CORS(app)


In [25]:
@app.route('/')
def hello():
    print('args:',request.args)
    response = make_response(jsonify({'result':'hello world!'}))
    return response



In [26]:
#注册，返回成功与否
@app.route('/api/register',methods=['POST',])
def process_register():
    username = request.form['username']
    password = request.form['password']
    msg = register(username,password)
    
    result = {'msg':msg}
    return make_response(jsonify(result))
#登录，返回user_id
@app.route('/api/login',methods=['POST',])
def process_login():
    username = request.form['username']
    password = request.form['password']
    user_id = login(username,password)
    
    result = {'user_id':user_id}
    return make_response(jsonify(result))
#更改昵称和头像
@app.route('/api/change_nickname',methods=['POST',])
def change_nickname():
    username = request.form['username']
    nickname = request.form['nickname']
    update_nickname(username,nickname)
    result = {'msg':'suc'}
@app.route('/api/change_avatar',methods=['POST',])
def change_avatar():
    username = request.form['username']
    f = request.files['file']
    save_path = os.path.join('./dist/images',secure_filename(f.filename))  #注意：没有的文件夹一定要先创建，不然会提示没有该路径
    f.save(save_path)
    avatar_path = os.path.join('images',secure_filename(f.filename)) 
    update_avatar(username,avatar_path)
    result = {'msg':'suc'}
    return make_response(jsonify(result))


In [27]:

@app.route('/api/get_user_info',methods=['GET',])
def get_user_info():
    username = request.args.get('username')
    result = {'avatar':'images/default.jpg',
              'nickname':username}
    item = get_user_by_name(username)
    if item:
        result['avatar'] = item[0][4]
        result['nickname'] = item[0][2]
    return make_response(jsonify(result))

In [28]:
#建立聊天室，返回聊天室id
@app.route('/api/create_chatroom',methods=['POST',])
def process_create_chatroom():
    user_id = request.form['user_id']
    room_id = create_chatroom(user_id)
    result = {'room_id':room_id}
    return make_response(jsonify(result))
#加入聊天室，返回成功与否
@app.route('/api/join_chatroom',methods=['POST',])
def process_join_chatroom():
    room_id = request.form['room_id']
    user_id = request.form['user_id']
    
    join_chatroom(room_id,user_id)
    
    result = {'msg':'suc'}
    return make_response(jsonify(result))

In [35]:
#发送聊天信息，返回成功与否
@app.route('/api/chat_in_chatroom',methods=['POST',])
def process_chat_in_chatroom():
    user_id = request.form['room_id']
    room_id = request.form['user_id']
    content = request.form['content']
    print('requset:',request.args)
    print('A!B!C!',user_id,room_id,content)
    chat_in_chatroom(room_id,user_id,content)
    result = {'msg':'suc'}
    return make_response(jsonify(result))
#刷新聊天信息，返回start_time到刷新时刻过程中的所有内容
@app.route('/api/refresh_chat',methods=['GET',])
def process_refresh_chat():
    target_id = request.args.get('target_id')
    target_type = request.args.get('target_type')
    start_time = request.args.get('start_time')
    print(target_id,target_type,start_time)
    chat_content,end_time = refresh_chat(target_id,target_type,start_time)
    result = {'chat_content':chat_content,
              'end_time':end_time}
    return make_response(jsonify(result))
    

In [23]:
show_chat_history_table()

((1, 2, 8368, 'chatroom', 'nihao cmq', datetime.datetime(2020, 6, 14, 17, 18)),
 (2, 2, 8368, 'chatroom', 'hei!', datetime.datetime(2020, 6, 14, 17, 19)),
 (3, 2333, 1, 'chatroom', 'qqq', datetime.datetime(2020, 6, 14, 22, 3)))

In [22]:
chat_in_chatroom(2333,1,'qqq')

In [17]:
refresh_chat(8368,'chatroom','2020-01-01')

([['wjs', 'nihao cmq', '2020-06-14 17:18:00'],
  ['wjs', 'hei!', '2020-06-14 17:19:00']],
 datetime.datetime(2020, 6, 14, 22, 0, 18, 78456))

In [18]:
@app.route('/api/upload',methods=['POST',])
def upload():
    f = request.files['file']
    upload_path = os.path.join('./files',secure_filename(f.filename))  #注意：没有的文件夹一定要先创建，不然会提示没有该路径
    f.save(upload_path)
    result = {'file_name':upload_path}
    return make_response(jsonify(result))

In [36]:
app.run('0.0.0.0',7777)

 * Running on http://0.0.0.0:7777/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Jun/2020 22:18:42] "POST /api/chat_in_chatroom HTTP/1.1" 200 -


requset: ImmutableMultiDict([])
A!B!C! 8368 1 chen


127.0.0.1 - - [14/Jun/2020 22:18:56] "POST /api/chat_in_chatroom?user_id=2&room_id=2333&content=bbbb%21 HTTP/1.1" 400 -


 * Running on http://0.0.0.0:7777/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Jun/2020 23:01:38] "GET /images/1.jpg HTTP/1.1" 200 -


In [43]:
def encoding_utf8(x):
    if x.i
    return str(x,'UTF-8')
def encoding_chat_history(chat_history):
    return list(map(encoding_utf8,chat_history))
list(map(encoding_chat_history,r))

SyntaxError: invalid syntax (<ipython-input-43-311ce2e1a1e9>, line 2)

## DEBUG

In [168]:

from flask import Flask, request

In [178]:
app = Flask(__name__,
            static_folder = "./dist/static",
            template_folder = "./dist")
CORS(app)

In [179]:
@app.route('/')
def hello():
    print('args:',request.args)
    return 'hello world!'

In [180]:
@app.route('/test1')
def test1():
    image_data = open('chat.jpeg', "rb").read()
    response = make_response(image_data)
    response.headers['Content-Type'] = 'image/png'
    return response

In [181]:
app.run('0.0.0.0',7777)

 * Running on http://0.0.0.0:7777/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Jun/2020 16:39:59] "GET /test1 HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2020 16:40:34] "GET /test1?name=chen HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2020 16:42:21] "GET /test1?name=x HTTP/1.1" 200 -
